###  
<!DOCTYPE html>
<html>
    <head>
        <title>
            <h1> Image Classification models </h1>
        </title>
        <body>
            <p> For Building <b> Image Classification models </b> we will go over the following options: <p><br/>
            <li> Training a small network from scratch (as a baseline ) </li>
            <li> Using the bottleneck features of a pre-trained network </li>
            <li> Fine-tuning the top layers of pre-trained network </li>
        </body>
</html>


###  
<!DOCTYPE html>
<html>
    <head>
        <h1><b>Data pre-processing and Data Augmentation<b></h1>      
    </head>
</html>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt

In [ ]:
data_generator = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    #rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
image = load_img('dogs-vs-cats/train/cat.0.jpg')
# convert the image to numpy array data
x = img_to_array(image)
# this is a numpy array with shape (1, 3, 150, 150)
x = x.reshape((1,) + x.shape)

i = 0
for batch in data_generator.flow(x, batch_size = 1,
                                save_to_dir='preview', save_prefix='cat',
                                save_format='jpeg'):
    i+=1
    if i > 20:
        break # Break. Because of, Otherwise the generator would loop indefinitely

####  
<!DOCTYPE html>
<html>
    <head>
        <h1><b> Trainint a small convent from scratch : 80% accuracy </b></h1>
    </head>
</html>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as k

In [ ]:
# # Dimension our image datasets
# image_width, image_height = 150, 150

# train_data_dir = 'dogs-vs-cats/train'
# validation_data_dir = 'dogs-vs-cats/test1'

# np_train_sampels = 2000
# np_test_samples = 800

# epochs = 50
# batch_size = 16


# if k.image_data_format() == 'channels_first':
#     input_shape = (3, image_width, image_height)
# else:
#     input_shape = (image_width, image_height, 3)
    

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape = input_shape))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss = 'binary_crossentropy',
#              optimizer = 'rmsprop',
#              matrics = ['accuracy'])


# train_datagen = ImageDataGenerator(rescale=1./255,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True)
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directiory( train_data_dir,
#                                                  target_size = (image_width, image_height),
#                                                  batch_size= batch_size,
#                                                  class_mode = 'binary')


# validation_generatior = test_datagen.flow_from_directiory( validation_data_dir, 
#                                                          target_size = (image_width, image_height),
#                                                          batch_size = batch_size,
#                                                          class_mode='binary')

# model.fit_generator(train_generator, 
#                    steps_per_epoch = np_train_sampels//batch_size,
#                    epochs = epochs,
#                    validation_data = validation_generatior,
#                    validation_steps =np_test_samples//batch_size)


# model.save_weights('first_try.h5')


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import numpy as np

K.clear_session()


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#model = Sequential()
model = Sequential()

model.add(Conv2D(32, (3, 3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 13496 images belonging to 2 classes.
Found 11504 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 89s 713ms/step - loss: 0.7122 - accuracy: 0.5140 - val_loss: 0.6767 - val_accuracy: 0.6325
Epoch 2/50
125/125 [==============================] - 86s 688ms/step - loss: 0.6703 - accuracy: 0.6290 - val_loss: 0.6373 - val_accuracy: 0.6587
Epoch 3/50
125/125 [==============================] - 86s 685ms/step - loss: 0.6356 - accuracy: 0.6365 - val_loss: 0.6162 - val_accuracy: 0.6600
Epoch 4/50
125/125 [==============================] - 86s 691ms/step - loss: 0.6155 - accuracy: 0.6645 - val_loss: 0.6339 - val_accuracy: 0.6438
Epoch 5/50
125/125 [==============================] - 87s 698ms/step - loss: 0.6146 - accuracy: 0.6780 - val_loss: 0.6094 - val_accuracy: 0.6825
Epoch 6/50
125/125 [==============================] - 87s 692ms/step - loss: 0.6209 - accuracy: 0.6835 - val_loss: 0.5375 - val_accuracy: 0.7225
Epoch 7/50
125/125 [========================